# Bitcoin Volatility Analysis - Data Exploration

This notebook provides an interactive exploration of the Bitcoin price data and market events.

## Objectives:
- Load and examine the raw data
- Explore Bitcoin price patterns
- Analyze volatility characteristics
- Visualize key market events


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Set up plotting style
plt.style.use('default')
sns.set_palette("husl")

# Get project root
project_root = Path('..')
print(f"Project root: {project_root.absolute()}")


ModuleNotFoundError: No module named 'seaborn'

In [ ]:
# Load Bitcoin price data
btc_data = pd.read_csv(project_root / 'data' / 'raw' / 'bitcoin_prices.csv', 
                       parse_dates=['Date'], index_col='Date')

print(f"Bitcoin data shape: {btc_data.shape}")
print(f"Date range: {btc_data.index.min()} to {btc_data.index.max()}")
print(f"\nColumns: {list(btc_data.columns)}")
btc_data.head()


In [ ]:
# Load market events data
events_data = pd.read_csv(project_root / 'data' / 'processed' / 'market_events.csv', 
                          parse_dates=['date'])

print(f"Events data shape: {events_data.shape}")
print(f"\nColumns: {list(events_data.columns)}")
events_data.head(10)
